# Capstone 2

## Polish Bankruptcy

## Modeling

In the pre-processing notebook, the logistic regression had high accuracy in predicting bankruptcy but resulted in very low recall.  In this notebook, I will apply resampling techniques to improve recall.  I will use both oversampling the minority class and undersampling the majority class on the logistic regression model and apply two other models with and without both resampling methods.  I will then compare the results from the classification reports for each process and evaluate the results. 

## Import Statements

In [1]:
#import all needed packages

import pandas as pd
import numpy as np
import os
import pickle

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn import __version__ as sklearn_version
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

## Load Data

In [2]:
# load the train test files from disk
filename = 'X_train_scaled.csv'
X_train_scaled = pd.read_csv('X_train_scaled.csv', index_col=0)
X_test_scaled = pd.read_csv('X_test_scaled.csv', index_col=0)
y_train = pd.read_csv('y_train.csv', index_col=0)
y_test = pd.read_csv('y_test.csv', index_col=0)

## View Train Test Data

In [3]:
#print scaled data
print(X_train_scaled.head())

            X03       X04       X06       X08       X09       X10       X11  \
12113 -0.163551 -0.019446 -0.213395 -0.025616 -0.037721 -0.073549 -0.339471   
3289   0.028093 -0.014577  0.010597 -0.019986 -0.035235  0.009176 -0.032724   
16567 -0.039875 -0.018477  0.031222 -0.023735 -0.034676 -0.014012 -0.033221   
21367  0.046737 -0.012777  0.045826 -0.023654 -0.031725 -0.011934 -0.003059   
32394  0.085720 -0.000891  0.058324 -0.010024 -0.030893  0.020583  0.006298   

            X12       X13       X14  ...       X50       X51       X52  \
12113 -0.025469 -0.012541 -0.257295  ... -0.017743  0.156218 -0.010243   
3289  -0.014684 -0.011254 -0.017094  ... -0.013072 -0.040386 -0.010977   
16567 -0.015058 -0.011162 -0.017484  ... -0.017177 -0.007341 -0.010657   
21367 -0.008565 -0.010231  0.006135  ... -0.015761 -0.045545 -0.010803   
32394  0.003409 -0.009519  0.013462  ...  0.000051 -0.066595 -0.010934   

            X53       X54       X56       X58       X62       X63      Year  
12

In [4]:
#print scaled test data
print(X_test_scaled.head())

            X03       X04       X06       X08       X09       X10       X11  \
25162 -0.025318 -0.018181  0.010845 -0.022973 -0.035122 -0.013729 -0.035424   
15362  0.047476 -0.012828  0.027407 -0.019149 -0.010805  0.011902  0.046414   
4236   0.025213 -0.014726  0.058733 -0.019909 -0.032972  0.009232 -0.006364   
33893 -0.023946 -0.017795 -0.044722 -0.024459 -0.029464 -0.024780 -0.019124   
40333 -0.003534 -0.016555  0.005887 -0.019379 -0.016365  0.011394 -0.033572   

            X12       X13       X14  ...       X50       X51       X52  \
25162 -0.015121 -0.011138 -0.019208  ... -0.017399 -0.046980 -0.010952   
15362  0.000817 -0.009838  0.043705  ... -0.011395 -0.044685 -0.010916   
4236  -0.009877 -0.010377  0.003547  ... -0.013215 -0.041003 -0.010829   
33893 -0.014367 -0.010980 -0.006695  ... -0.016157  0.045150 -0.008629   
40333 -0.014896 -0.011003 -0.018258  ... -0.015099 -0.044829 -0.010918   

            X53       X54       X56       X58       X62       X63      Year  
25

In [5]:
#print y train data
print(y_train.head())

       Class
12113      0
3289       0
16567      0
21367      0
32394      0


In [6]:
#print y test data
print(y_test)

       Class
25162      0
15362      0
4236       0
33893      0
40333      0
...      ...
10939      0
9520       0
41847      0
38614      0
33968      0

[10852 rows x 1 columns]


## Create Logistic Regression object

In [7]:
# create the regression object
clf1 = LogisticRegression(max_iter=1000)
# fit the scaled and training data
clf1.fit(X_train_scaled, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression(max_iter=1000)

In [8]:
# run the regression model
y_te_pred_lr = clf1.predict(X_test_scaled)
y_predict_training_lr = clf1.predict(X_train_scaled)

## Print Logistic Accuracy

In [9]:
# print test accuracy
print("\n")
print("[Test] Accuracy score: (y_test, y_te_pred_lr)",accuracy_score(y_test, y_te_pred_lr))

# print training accuracy
print("\n")
print("[Training] Accuracy score: (y_train, y_predict_training_lr)",accuracy_score(y_train, y_predict_training_lr))



[Test] Accuracy score: (y_test, y_te_pred_lr) 0.9517139697751567


[Training] Accuracy score: (y_train, y_predict_training_lr) 0.9518631155346665


## Print Logistic Classification

In [10]:
# print classification scores  0 is non bankrupt entities 1 is bankrupt entitiies
print("[Training Classification Report]")
print(classification_report(y_train, y_predict_training_lr))

print("[Test Classification Report]")
print(classification_report(y_test, y_te_pred_lr))

[Training Classification Report]
              precision    recall  f1-score   support

           0       0.95      1.00      0.98     30985
           1       0.52      0.01      0.01      1568

    accuracy                           0.95     32553
   macro avg       0.74      0.50      0.49     32553
weighted avg       0.93      0.95      0.93     32553

[Test Classification Report]
              precision    recall  f1-score   support

           0       0.95      1.00      0.98     10329
           1       0.44      0.01      0.02       523

    accuracy                           0.95     10852
   macro avg       0.70      0.50      0.50     10852
weighted avg       0.93      0.95      0.93     10852



## Over sample with smote 

In [11]:
smote_lr = SMOTE(sampling_strategy='minority')
X_sm_train_lr2, y_sm_train_lr2 = smote_lr.fit_resample(X_train_scaled, y_train)

## Logistic Regression on Resampled data

In [12]:
clf2 = LogisticRegression(max_iter=1000)
# fit the scaled and training data
clf2.fit(X_sm_train_lr2, y_sm_train_lr2)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression(max_iter=1000)

In [13]:
y_te_pred_lr2 = clf2.predict(X_test_scaled)

y_predict_training_lr2 = clf2.predict(X_sm_train_lr2)
#y_predict_training_xgb = xgb_model1.predict(X_train_scaled)

#y_te_pred_lr = clf1.predict(X_test_scaled)
#y_predict_training_lr = clf1.predict(X_train_scaled)


## Print accuracy on resampled data

In [14]:
# print test accuracy
print("\n")
print("[Test] Accuracy score: (y_test, y_te_pred_lr)",accuracy_score(y_test, y_te_pred_lr2))

# print training accuracy
#print("\n")
#print("[Training] Accuracy score: (y_train, y_predict_training_lr)",accuracy_score(y_train, y_predict_training_lr2))
#print(y_train)
print(y_predict_training_lr2.shape)



[Test] Accuracy score: (y_test, y_te_pred_lr) 0.6872465904902322
(61970,)


## Create XGB Object

In [15]:
xgb_model1 = XGBClassifier()
xgb_model1.fit(X_train_scaled, y_train)

y_te_pred_xgb = xgb_model1.predict(X_test_scaled)
y_predict_training_xgb = xgb_model1.predict(X_train_scaled)


accuracy = accuracy_score(y_test, y_te_pred_xgb)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[11:49:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 95.33%


##  Print XGB Classification Report

In [16]:

# print classification scores  0 is non bankrupt entities 1 is bankrupt entitiies
print("[Training Classification Report]")
print(classification_report(y_train, y_predict_training_xgb))

print("[Test Classification Report]")
print(classification_report(y_test, y_te_pred_xgb))

[Training Classification Report]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     30985
           1       1.00      0.83      0.91      1568

    accuracy                           0.99     32553
   macro avg       1.00      0.92      0.95     32553
weighted avg       0.99      0.99      0.99     32553

[Test Classification Report]
              precision    recall  f1-score   support

           0       0.96      0.99      0.98     10329
           1       0.55      0.18      0.27       523

    accuracy                           0.95     10852
   macro avg       0.75      0.59      0.62     10852
weighted avg       0.94      0.95      0.94     10852



##  Resample data with SMOTE

In [17]:
smote_xgb = SMOTE(sampling_strategy='minority')
X_sm_train_xgb, y_sm_train_xgb = smote_xgb.fit_resample(X_train_scaled, y_train)

## Create SMOTE Object with resampled data

In [18]:
xgb_model2 = XGBClassifier()
xgb_model2.fit(X_sm_train_xgb, y_sm_train_xgb)

y_te_pred_smote = xgb_model2.predict(X_test_scaled)
y_predict_training_smote = xgb_model2.predict(X_sm_train_xgb)


accuracy = accuracy_score(y_test, y_te_pred_smote)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[11:49:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 91.49%


## Run Classification Report on resampled data

In [19]:
# print classification scores  0 is non bankrupt entities 1 is bankrupt entitiies
print("[Training Classification Report]")
print(classification_report(y_sm_train_xgb, y_predict_training_smote))

print("[Test Classification Report]")
print(classification_report(y_test, y_te_pred_smote))

[Training Classification Report]
              precision    recall  f1-score   support

           0       0.99      0.97      0.98     30985
           1       0.97      0.99      0.98     30985

    accuracy                           0.98     61970
   macro avg       0.98      0.98      0.98     61970
weighted avg       0.98      0.98      0.98     61970

[Test Classification Report]
              precision    recall  f1-score   support

           0       0.98      0.93      0.95     10329
           1       0.31      0.62      0.41       523

    accuracy                           0.91     10852
   macro avg       0.64      0.77      0.68     10852
weighted avg       0.95      0.91      0.93     10852

